In [1]:

from getpass import getpass
from typing import List
from ase import Atoms, Atom
from langchain.agents import tool
from ase.build import bulk
from ase.data import reference_states, atomic_numbers
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from typing import Callable, List

from langchain.memory import ConversationBufferMemory
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
)

from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.pydantic_v1 import BaseModel
from langchain.agents import AgentExecutor
from pymatgen.io.ase import AseAtomsAdaptor
from mp_api.client import MPRester
import os, sys, glob

from ase.calculators.emt import EMT
from ase.eos import calculate_eos
from ase.units import kJ
from ase.lattice.cubic import FaceCenteredCubic
from ase import Atoms
import io
from ase.io import write, read
import numpy as np
import ast

from ase.calculators.espresso import Espresso, EspressoProfile


/opt/anaconda3/envs/agent/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:

class AtomsDict(BaseModel):
    numbers: List[int]
    positions: List[List[float]]
    cell: List[List[float]]
    pbc: List[bool]
def structure(concentration: float) -> AtomsDict:
    concentration = 0.5
    atoms = FaceCenteredCubic("Cu", latticeconstant=3.58)
    atoms *= (1,1,2)
    # Calculate the number of Cu atoms to replace
    num_atoms_to_replace = int(concentration * len(atoms))
    # Randomly select indices to replace
    indices_to_replace = np.random.choice(len(atoms), num_atoms_to_replace, replace=False)
    atoms.numbers[indices_to_replace] = 79
    scaleFactor = concentration * (6.5 - 3.58) / 3.58 + 1
    atoms.set_cell(atoms.cell * scaleFactor, scale_atoms=True)
    return atoms.todict()

In [9]:
print(structure(0.5))

{'numbers': array([79, 29, 29, 79, 29, 79, 79, 29]), 'positions': array([[0.  , 0.  , 0.  ],
       [2.52, 2.52, 0.  ],
       [2.52, 0.  , 2.52],
       [0.  , 2.52, 2.52],
       [0.  , 0.  , 5.04],
       [2.52, 2.52, 5.04],
       [2.52, 0.  , 7.56],
       [0.  , 2.52, 7.56]]), 'cell': array([[ 5.04,  0.  ,  0.  ],
       [ 0.  ,  5.04,  0.  ],
       [ 0.  ,  0.  , 10.08]]), 'pbc': array([ True,  True,  True])}


In [1]:
content = "&CONTROL\n    calculation = 'scf'\n    restart_mode = 'from_scratch'\n    pseudo_dir = './'\n    outdir = './'\n    prefix = 'cu_au_alloy'\n    tprnfor = .true.\n    tstress = .true.\n/\n&SYSTEM\n    ibrav = 0\n    nat = 8\n    ntyp = 2\n    ecutwfc = 60.0\n    ecutrho = 480.0\n    occupations = 'smearing'\n    smearing = 'mp'\n    degauss = 0.02\n/\n&ELECTRONS\n    conv_thr = 1.0e-7\n    mixing_beta = 0.3\n/\nATOMIC_SPECIES\nCu 63.546 Cu.pbe-dn-kjpaw_psl.1.0.0.UPF\nAu 196.967 Au.pbe-n-kjpaw_psl.1.0.0.UPF\n\nK_POINTS automatic\n5 5 3 0 0 0\n\nATOMIC_POSITIONS crystal\nCu  0.000000000   0.000000000   0.000000000\nCu  0.500000000   0.500000000   0.000000000\nCu  0.500000000   0.000000000   0.250000000\nCu  0.000000000   0.500000000   0.250000000\nAu  0.000000000   0.000000000   0.500000000\nAu  0.500000000   0.500000000   0.500000000\nAu  0.500000000   0.000000000   0.750000000\nAu  0.000000000   0.500000000   0.750000000\n\nCELL_PARAMETERS angstrom\n 5.040000000   0.000000000   0.000000000\n 0.000000000   5.040000000   0.000000000\n 0.000000000   0.000000000  10.080000000"
print(content)

&CONTROL
    calculation = 'scf'
    restart_mode = 'from_scratch'
    pseudo_dir = './'
    outdir = './'
    prefix = 'cu_au_alloy'
    tprnfor = .true.
    tstress = .true.
/
&SYSTEM
    ibrav = 0
    nat = 8
    ntyp = 2
    ecutwfc = 60.0
    ecutrho = 480.0
    occupations = 'smearing'
    smearing = 'mp'
    degauss = 0.02
/
&ELECTRONS
    conv_thr = 1.0e-7
    mixing_beta = 0.3
/
ATOMIC_SPECIES
Cu 63.546 Cu.pbe-dn-kjpaw_psl.1.0.0.UPF
Au 196.967 Au.pbe-n-kjpaw_psl.1.0.0.UPF

K_POINTS automatic
5 5 3 0 0 0

ATOMIC_POSITIONS crystal
Cu  0.000000000   0.000000000   0.000000000
Cu  0.500000000   0.500000000   0.000000000
Cu  0.500000000   0.000000000   0.250000000
Cu  0.000000000   0.500000000   0.250000000
Au  0.000000000   0.000000000   0.500000000
Au  0.500000000   0.500000000   0.500000000
Au  0.500000000   0.000000000   0.750000000
Au  0.000000000   0.500000000   0.750000000

CELL_PARAMETERS angstrom
 5.040000000   0.000000000   0.000000000
 0.000000000   5.040000000   0.000000